Batch Adding Users from a CSV file.
Let's first confirm who the existing users are:

In [22]:
from config import *

# Function to fetch and print existing users with user ID, email, and last updated
def get_existing_users():
    response = requests.get(f"{base_url}/users?limit=9999", headers=headers_get)
    if response.status_code == 200:
        users = response.json().get('data', [])
        if not users:
            print("No users found.")
            return {}

        user_dict = {}
        for user in users:
            user_id = user.get('id', 'N/A')
            email = user.get('email', 'N/A')
            updated_at = user.get('updatedAt', None)

            # Convert updatedAt (if exists) to human-readable format
            if updated_at:
                updated_at = datetime.fromtimestamp(updated_at / 1000).strftime('%Y-%m-%d %H:%M:%S')
            else:
                updated_at = 'Unknown'

            print(f"User ID: {user_id}, Last Updated: {updated_at}, Email: {email}")
            user_dict[email] = {'id': user_id, 'last_updated': updated_at}
        
        return user_dict
    else:
        print(f"Failed to fetch users: {response.status_code}")
        return {}

# Get the full user dictionary
users_dict = get_existing_users()

# Extract emails as a list
email_list = list(users_dict.keys())

print(f'\nThere are a total of {len(email_list)} users at present.')

User ID: db1a4050-32c9-468b-b53f-7c370bb6a612, Last Updated: 2024-07-26 14:03:01, Email: abigail.michel@gov.bc.ca
User ID: 10e6f061-4506-42bc-9bde-3606ade95664, Last Updated: 2024-11-28 14:27:13, Email: adam.dewey@gov.bc.ca
User ID: c6573e36-7219-4838-94ad-63a0c5f760a8, Last Updated: 2024-04-09 11:48:49, Email: admin@open-metadata.org
User ID: 7cbdf440-f954-44a4-874e-2d6d9222c7ab, Last Updated: 2024-11-28 14:27:20, Email: adria.fradley@gov.bc.ca
User ID: afa0a048-78bc-4b90-9b61-06164669c78d, Last Updated: 2024-11-28 15:49:15, Email: adrienne.canty@gov.bc.ca
User ID: bb152deb-2f31-492e-beb6-3e049a9219ef, Last Updated: 2024-11-28 15:49:00, Email: albert.law@gov.bc.ca
User ID: cebc9290-88a5-429f-b3c6-93d2e2ad3d40, Last Updated: 2024-11-28 14:27:19, Email: alec.dale@gov.bc.ca
User ID: db7ad454-143b-4b9c-b148-b5dc28c702f8, Last Updated: 2024-08-21 12:42:36, Email: alex.nishta@gov.bc.ca
User ID: 29e164d9-67fa-4b42-8bd2-708d3836a293, Last Updated: 2024-11-28 15:49:16, Email: allan.lidstone@go

In [9]:
# Go ahead and import the CSV data to then upload new users to OMD
df_users = pd.read_csv('reference_csvs/nov2024_custodians.csv', header=1)
df_users.head()

,Application,Status,Custodian 1,Custodian 2
0,AACDET,Retired,"Nussbaum, Albert",Albert.Nussbaum@gov.bc.ca
1,ACAT,Active,"Tesch, David",David.Tesch@gov.bc.ca
2,ACRFD,Active,"Yetskalo, Valentina",Valentina.Yetskalo@gov.bc.ca
3,ACS,Active,"Searle, Barb",Barb.Searle@gov.bc.ca
4,ADAM (FOR),Active,"Dewey, Adam",Adam.Dewey@gov.bc.ca


We should assign a user a role, in this excercise we know that we are dealing with Data Custodians so finding the role ID for that is our target.

In [8]:
def get_role_id_by_name(role_name):
    """Fetch the role ID by its name."""
    response = requests.get(f"{base_url}/roles?limit=9999", headers=headers_get)
    if response.status_code == 200:
        roles = response.json().get('data', [])
        for role in roles:
            if role['name'] == role_name:
                return role['id']  # Return the ID of the matching role
        print(f"Role '{role_name}' not found.")
    else:
        print(f"Failed to fetch roles: {response.status_code}")
        print(response.json())
    return None

# Example Usage
role_name = "NRM Data Custodian Role"
role_id = get_role_id_by_name(role_name)

if role_id:
    print(f"Role ID for '{role_name}' is: {role_id}")
else:
    print(f"Could not find Role ID for '{role_name}'.")


Role ID for 'NRM Data Custodian Role' is: b35939ef-5e7b-4eee-a2ef-94de939e6c3c


In [16]:
def add_user(email, role_id):

    email = email.strip()
    
    user = {
        "email": email,
        "displayName":email.split("@")[0].replace(".", " ").title(),
        "name":email.split("@")[0],
        "roles": [role_id] #Multiple roles are possible, but let's just use 1 for now
    }

    # Send POST request to add the user
    response = requests.post(
        f"{base_url}/users",
        headers=headers_get,
        json=user
    )

    if response.status_code in [200,201]:
        print(f"ID: {response.json().get('id')} - User {user['displayName']} added successfully.")
    else:
        print(f"Failed to add user {user['displayName']}: {response.status_code}")
        print(response.json())

In [23]:
#Add 1 user as a test
add_user(df_users['Custodian 2'][0],role_id)

ID: 337775ff-1062-43fd-90cb-ea23dbe4a056 - User Albert Nussbaum added successfully.


In [24]:
# Add users
for user_email in df_users['Custodian 2'].unique():  # Iterate directly over the column values
    if user_email not in email_list:
        # Use the row index to fetch associated username if needed
        user_row = df_users[df_users['Custodian 2'] == user_email].index[0]  # Find the row index
        add_user(user_email, role_id)  # Pass user_email and username
    else:
        print(f"User {user_email} already exists.")

ID: b73df4cf-1c16-43a0-b16b-2cf131ed463b - User Albert Nussbaum added successfully.
ID: da12b2c2-d17f-44d5-80c1-28994d245a05 - User David Tesch added successfully.
ID: 756e4dcd-fcb7-430a-b312-91a32c41dfcd - User Valentina Yetskalo added successfully.
ID: c777f1fa-c422-4c26-b351-21fff7abec69 - User Barb Searle added successfully.
Failed to add user Adam Dewey: 409
{'code': 409, 'message': 'Entity already exists'}
Failed to add user Dylan Labh: 409
{'code': 409, 'message': 'Entity already exists'}
Failed to add user Diana Cooper: 409
{'code': 409, 'message': 'Entity already exists'}
Failed to add user Stephen Maciver: 409
{'code': 409, 'message': 'Entity already exists'}
User jessica.schafer@gov.bc.ca already exists.
Failed to add user Tarik Dessouki: 409
{'code': 409, 'message': 'Entity already exists'}
Failed to add user Ted Zimmerman: 409
{'code': 409, 'message': 'Entity already exists'}
User yao.cui@gov.bc already exists.
Failed to add user Krista Gunnarsen: 409
{'code': 409, 'messag

KeyboardInterrupt: 

In theory, you should be able to assign multiple user roles. 

Below the code snippet is assigning roles to users that had no assinged roles, it has not been tested on users with already existing roles**

In [11]:
def get_user_id_by_email(email):
    """
    Fetches the user ID associated with a given email.
    """
    response = requests.get(f"{base_url}/users?limit=9999", headers=headers_get)
    if response.status_code != 200:
        print(f"Failed to fetch users: {response.status_code}")
        print(response.text)
        return None

    users = response.json().get("data", [])
    for user in users:
        if user["email"].strip().lower() == email.strip().lower():
            return user["id"]
    
    print(f"User with email {email} not found.")
    return None


def patch_user_roles(email, role_id):
    """
    Update a user's roles by adding a specific role ID using the PATCH method.
    """
    user_id = get_user_id_by_email(email)
    if not user_id:
        return

    patch_payload = [
        {
            "op": "add",
            "path": "/roles/-",  # Append role to array
            "value": {"id": role_id}
        }
    ]

    response = requests.patch(
        f"{base_url}/users/{user_id}",
        headers=headers_patch,
        json=patch_payload
    )

    if response.status_code in [200, 204]:
        print(f"Role {role_id} added successfully to user {email}.")
    else:
        print(f"Failed to update roles for {email}: {response.status_code}")
        try:
            print("Response:", response.json())
        except ValueError:
            print("Response text:", response.text)


for user_email in df_users['Custodian 2'].unique():
    patch_user_roles(user_email, role_id)
    time.sleep(1)

Role 74ed83ea-17bd-4b91-a81e-2ecf67d4ee81 added successfully to user Albert.Nussbaum@gov.bc.ca.
Role 74ed83ea-17bd-4b91-a81e-2ecf67d4ee81 added successfully to user David.Tesch@gov.bc.ca .
Role 74ed83ea-17bd-4b91-a81e-2ecf67d4ee81 added successfully to user Valentina.Yetskalo@gov.bc.ca.
Role 74ed83ea-17bd-4b91-a81e-2ecf67d4ee81 added successfully to user Barb.Searle@gov.bc.ca.
Role 74ed83ea-17bd-4b91-a81e-2ecf67d4ee81 added successfully to user Adam.Dewey@gov.bc.ca.
Role 74ed83ea-17bd-4b91-a81e-2ecf67d4ee81 added successfully to user Dylan.Labh@gov.bc.ca.
Role 74ed83ea-17bd-4b91-a81e-2ecf67d4ee81 added successfully to user Diana.Cooper@gov.bc.ca.
Role 74ed83ea-17bd-4b91-a81e-2ecf67d4ee81 added successfully to user Stephen.MacIver@gov.bc.ca.
Role 74ed83ea-17bd-4b91-a81e-2ecf67d4ee81 added successfully to user jessica.schafer@gov.bc.ca.
Role 74ed83ea-17bd-4b91-a81e-2ecf67d4ee81 added successfully to user Tarik.Dessouki@gov.bc.ca.
Role 74ed83ea-17bd-4b91-a81e-2ecf67d4ee81 added successful